# Lean 4 - Quantificateurs et Logique du Premier Ordre

## Introduction

Ce notebook etend la logique propositionnelle du notebook precedent a la **logique du premier ordre** en introduisant les **quantificateurs** : le quantificateur universel ("pour tout") et le quantificateur existentiel ("il existe").

Ces quantificateurs permettent d'exprimer des proprietes sur des **ensembles infinis** d'objets, ce qui est essentiel pour les mathematiques.

### Objectifs pedagogiques

1. Maitriser le quantificateur universel `forall` et son utilisation
2. Maitriser le quantificateur existentiel `Exists` et la construction de temoins
3. Manipuler les proprietes arithmetiques sur `Nat`
4. Utiliser les hypotheses anonymes et les references
5. Comprendre l'interaction entre quantificateurs

### Prerequis

- Notebooks **Lean-1**, **Lean-2** et **Lean-3** completes
- Familiarite avec les concepts de base de la logique du premier ordre

### Duree estimee : 40-45 minutes

---

## Rappel : Types Dependants et Pi-types

En Lean, les quantificateurs sont fondes sur les **types dependants** (Pi-types) que nous avons vus au notebook 2 :

- `(x : A) -> B x` est le type des fonctions qui prennent `x : A` et retournent une valeur de type `B x` (qui peut dependre de `x`)
- Quand `B x : Prop`, ce Pi-type represente le quantificateur universel "pour tout x de type A, B(x)"

## 1. Le Quantificateur Universel `forall`

### 1.1 Definition et syntaxe

Le quantificateur universel `forall x : A, P x` (note aussi `\forall x : A, P x` ou avec le symbole Unicode `∀`) exprime que la propriete `P` est vraie pour **tout** element `x` du type `A`.

En Lean, `forall` est simplement un alias pour le Pi-type dans `Prop`.

### Saisie des caracteres Unicode dans Lean

Dans VSCode avec l'extension Lean 4, les symboles Unicode se tapent avec des raccourcis :

| Symbole | Raccourci | Description |
|---------|-----------|-------------|
| `∀` | `\forall` ou `\all` | Quantificateur universel |
| `∃` | `\exists` ou `\ex` | Quantificateur existentiel |
| `→` | `\to` ou `\r` | Implication (fleche droite) |
| `←` | `\l` | Fleche gauche |
| `↔` | `\iff` | Equivalence |
| `∧` | `\and` | Conjonction |
| `∨` | `\or` | Disjonction |
| `¬` | `\not` ou `\neg` | Negation |
| `⟨` `⟩` | `\<` `\>` | Angle brackets |
| `▸` | `\t` | Triangle (substitution) |
| `λ` | `\lam` | Lambda |

**Conseil** : Dans un notebook Jupyter avec le kernel lean4, vous pouvez aussi utiliser les mots-cles ASCII (`forall`, `exists`, `->`, `<->`, `/\`, `\/`).

In [ ]:
-- Syntaxes equivalentes pour le quantificateur universel
variable (A : Type) (P : A -> Prop)

#check forall x : A, P x     -- Prop
#check \forall x : A, P x    -- meme chose avec Unicode
#check (x : A) -> P x        -- Pi-type equivalent

-- Exemples concrets
#check forall n : Nat, n >= 0           -- "Tout naturel est >= 0"
#check forall n : Nat, n + 0 = n        -- "n + 0 = n pour tout n"

### 1.2 Introduction du forall

Pour prouver `forall x : A, P x`, on doit fournir une **fonction** qui, etant donne un `x : A` arbitraire, produit une preuve de `P x`.

C'est exactement comme programmer une fonction!

In [ ]:
-- Prouver : pour tout n, n + 0 = n
-- Introduction par lambda-abstraction
theorem add_zero_forall : forall n : Nat, n + 0 = n :=
  fun n => rfl   -- La preuve de n + 0 = n est rfl (calcul)

-- Syntaxe alternative avec fun ... =>
theorem add_zero_forall' : \forall n : Nat, n + 0 = n :=
  fun n : Nat => Nat.add_zero n   -- Utilise le lemme de la bibliotheque

#check add_zero_forall   -- add_zero_forall : forall n, n + 0 = n

### 1.3 Elimination du forall

Pour utiliser `forall x : A, P x`, on l'**applique** a une valeur concrete `a : A` pour obtenir `P a`.

C'est l'application de fonction!

In [ ]:
-- Si on a une preuve de forall, on peut l'instancier
theorem use_forall : 5 + 0 = 5 :=
  add_zero_forall 5   -- Appliquer le theoreme a 5

#eval 5 + 0  -- 5 (Lean calcule)

-- Exemple avec une propriete quelconque
variable (P : Nat -> Prop)
variable (h : forall n : Nat, P n)   -- Hypothese : P vrai pour tout n

-- On peut en deduire P 42
theorem instance_of_forall : P 42 := h 42

### 1.4 Proprietes avec plusieurs quantificateurs

Quand on a plusieurs variables quantifiees, on peut les introduire une par une ou en groupe. Les hypotheses intermediaires (`have`) permettent de structurer les preuves complexes.

In [ ]:
-- forall x y, P x y est equivalent a forall x, forall y, P x y
variable (P : Nat -> Nat -> Prop)

#check forall x y : Nat, P x y      -- Prop
#check forall x : Nat, forall y : Nat, P x y  -- meme chose

-- Prouver une propriete avec deux quantificateurs
theorem comm_add : forall x y : Nat, x + y = y + x :=
  fun x y => Nat.add_comm x y   -- Lemme de la bibliotheque

#check comm_add 3 5   -- 3 + 5 = 5 + 3 : Prop

## 2. Le Quantificateur Existentiel `Exists`

### 2.1 Definition et syntaxe

Le quantificateur existentiel `Exists (fun x => P x)` (note `\exists x, P x`) exprime qu'il **existe au moins un** element `x` tel que `P x` soit vrai.

Une preuve de `Exists` contient deux choses :
1. Un **temoin** : une valeur concrete `a` de type `A`
2. Une **preuve** que `P a` est vrai

In [ ]:
-- Syntaxes pour le quantificateur existentiel
variable (A : Type) (P : A -> Prop)

#check Exists P                 -- Prop
#check Exists (fun x : A => P x)  -- equivalent
#check \exists x : A, P x       -- notation

-- Exemples
#check \exists n : Nat, n > 5       -- "Il existe n > 5"
#check \exists n : Nat, n * n = 4   -- "Il existe n tel que n^2 = 4"

### 2.2 Introduction du Exists

Pour prouver `\exists x : A, P x`, on doit fournir :
- Un **temoin** `a : A`
- Une **preuve** de `P a`

On utilise la notation angle brackets `⟨temoin, preuve⟩` ou `Exists.intro`.

In [ ]:
-- Prouver : il existe n tel que n > 5
-- Temoin : 6, Preuve : 6 > 5
theorem exists_gt_5 : \exists n : Nat, n > 5 :=
  ⟨6, Nat.lt_succ_self 5⟩   -- 6 > 5 car 5 < 6 = succ 5

-- Version avec Exists.intro explicite
theorem exists_gt_5' : \exists n : Nat, n > 5 :=
  Exists.intro 6 (Nat.lt_succ_self 5)

-- Prouver : il existe n tel que n + n = 10
theorem exists_double_10 : \exists n : Nat, n + n = 10 :=
  ⟨5, rfl⟩   -- Temoin : 5, Preuve : 5 + 5 = 10 par calcul

### 2.3 Elimination du Exists

Pour utiliser une preuve de `\exists x : A, P x`, on fait une **analyse par cas** : on suppose qu'on a un `x` quelconque et une preuve de `P x`, puis on doit montrer le but.

C'est `Exists.elim` ou la syntaxe `match`/`let`.

In [ ]:
-- Si il existe n pair, alors il existe m tel que 2 * m existe
-- (exemple simplifie)

-- Elimination avec Exists.elim
theorem exists_elim_example
  (h : \exists n : Nat, n > 0) : True :=
  Exists.elim h (fun n hn => True.intro)

-- Elimination avec match (plus lisible)
theorem exists_elim_match
  (h : \exists n : Nat, n > 0) : \exists m : Nat, m >= 0 :=
  match h with
  | ⟨n, hn⟩ => ⟨n, Nat.zero_le n⟩

-- Syntaxe let ... := ... in ...
theorem exists_elim_let
  (h : \exists n : Nat, n > 0) : \exists m : Nat, m >= 0 :=
  let ⟨n, _⟩ := h
  ⟨n, Nat.zero_le n⟩

## 3. Proprietes Arithmetiques sur Nat

### 3.1 Lemmes de base

Lean fournit de nombreux lemmes sur les nombres naturels dans le namespace `Nat`.

In [ ]:
-- Quelques lemmes utiles
#check Nat.add_zero       -- n + 0 = n
#check Nat.zero_add       -- 0 + n = n
#check Nat.add_comm       -- n + m = m + n
#check Nat.add_assoc      -- (n + m) + k = n + (m + k)
#check Nat.mul_comm       -- n * m = m * n
#check Nat.mul_assoc      -- (n * m) * k = n * (m * k)
#check Nat.add_mul        -- (n + m) * k = n * k + m * k
#check Nat.mul_add        -- n * (m + k) = n * m + n * k

### 3.2 Preuves arithmetiques

Les preuves sur les nombres naturels utilisent souvent les lemmes de la bibliotheque standard (`Nat.add_comm`, `Nat.mul_assoc`, etc.) combines avec la reflexivite et la congruence.

In [ ]:
-- Preuve que n + 1 = 1 + n
theorem add_one_comm (n : Nat) : n + 1 = 1 + n :=
  Nat.add_comm n 1

-- Preuve avec calc
theorem calc_arith (a b c : Nat) : (a + b) + c = a + (b + c) :=
  calc (a + b) + c
       = a + (b + c) := Nat.add_assoc a b c

-- Preuve plus complexe
theorem arith_example (a b : Nat) : a + b + a = 2 * a + b :=
  calc a + b + a
       = a + (b + a)   := Nat.add_assoc a b a
     _ = a + (a + b)   := congrArg (a + ·) (Nat.add_comm b a)
     _ = (a + a) + b   := (Nat.add_assoc a a b).symm
     _ = 2 * a + b     := congrArg (· + b) (Nat.two_mul a).symm

## 4. Combinaison de Quantificateurs

L'ordre des quantificateurs est crucial : `forall x, exists y, P x y` (pour tout x il existe un y) est different de `exists y, forall x, P x y` (il existe un y pour tous les x).

In [ ]:
variable (A : Type) (P : A -> Prop)

-- De forall a exists (si A est habite)
theorem forall_to_exists (a : A) (h : forall x : A, P x) : \exists x : A, P x :=
  ⟨a, h a⟩

-- Negation de exists = forall not
theorem not_exists_iff : (\neg \exists x : A, P x) <-> forall x : A, \neg P x :=
  ⟨fun h x hp => h ⟨x, hp⟩,
   fun h ⟨x, hp⟩ => h x hp⟩

### 4.2 Ordre des quantificateurs

**Attention** : l'ordre des quantificateurs compte!

- `\forall x, \exists y, P x y` : pour chaque x, il existe un y (different selon x)
- `\exists y, \forall x, P x y` : il existe un y qui marche pour tous les x

In [ ]:
variable (P : Nat -> Nat -> Prop)

-- L'implication existe-forall vers forall-existe est toujours vraie
theorem exists_forall_to_forall_exists
  (h : \exists y : Nat, forall x : Nat, P x y) : forall x : Nat, \exists y : Nat, P x y :=
  fun x => match h with
    | ⟨y, hy⟩ => ⟨y, hy x⟩

-- L'inverse n'est pas vrai en general!
-- Exemple : P x y := x <= y
-- forall x, exists y, x <= y  -- Vrai : y = x marche
-- exists y, forall x, x <= y  -- Faux : aucun y ne majore tous les x

## 5. Hypotheses Anonymes et Nommage

### 5.1 La construction `have`

`have` permet d'introduire des lemmes intermediaires dans une preuve.

In [ ]:
-- have avec nom
theorem have_example (a b : Nat) (h : a = b) : b = a :=
  have h' : b = a := h.symm
  h'

-- have anonyme (utilise `this`)
theorem have_anon (a b : Nat) (h : a = b) : b = a :=
  have : b = a := h.symm
  this   -- Reference a la derniere hypothese anonyme

### 5.2 References avec `‹›`

La notation `‹P›` (guillemets francais) permet de faire reference a une hypothese du contexte de type `P` sans la nommer.

In [ ]:
-- Utilisation de ‹›
theorem bracket_example (p q : Prop) (hp : p) (hq : q) : p /\ q :=
  ⟨‹p›, ‹q›⟩   -- Reference les hypotheses par leur type

-- Plus complexe
theorem bracket_complex (p q r : Prop)
  (hpq : p -> q) (hqr : q -> r) : p -> r :=
  fun hp : p =>
    have : q := hpq ‹p›
    have : r := hqr ‹q›
    ‹r›

### 5.3 Suppositions avec `assume`/`fun`

En Lean 4, on utilise `fun` pour introduire une hypothese dans une preuve par termes. C'est equivalent a `assume` en Lean 3 ou `intro` en mode tactique.

In [ ]:
-- Les deux syntaxes sont equivalentes
theorem assume_example1 (p q : Prop) : p -> q -> p :=
  fun hp hq => hp

-- Avec annotation de type
theorem assume_example2 (p q : Prop) : p -> q -> p :=
  fun (hp : p) (hq : q) => hp

-- Avec destructuration
theorem assume_destruct (p q : Prop) : p /\ q -> q /\ p :=
  fun ⟨hp, hq⟩ => ⟨hq, hp⟩

## 6. Egalite et Quantificateurs

### 6.1 Réécriture avec l'égalité

In [ ]:
-- Substitution dans un forall
theorem subst_in_forall (P : Nat -> Prop) (a b : Nat)
  (h : a = b) (hp : forall n, P n) : P b :=
  hp b   -- Direct, pas besoin de h ici

-- Mais si on a P a et a = b, on peut avoir P b
theorem subst_prop (P : Nat -> Prop) (a b : Nat)
  (h : a = b) (hp : P a) : P b :=
  h ▸ hp   -- Substitution avec le triangle

### 6.2 Egalite fonctionnelle

In [ ]:
-- Deux fonctions sont egales si elles ont les memes valeurs
-- C'est l'extensionnalite fonctionnelle
#check @funext  -- funext : (forall x, f x = g x) -> f = g

-- Exemple
def f (n : Nat) : Nat := n + 0
def g (n : Nat) : Nat := n

theorem f_eq_g : f = g :=
  funext (fun n => Nat.add_zero n)

## 7. Exemples Complets

### 7.1 Proprietes de divisibilite

In [ ]:
-- Definition : a divise b
def divides (a b : Nat) : Prop := \exists k : Nat, b = a * k

notation:50 a " | " b => divides a b

-- Tout nombre divise 0
theorem divides_zero (a : Nat) : a | 0 :=
  ⟨0, (Nat.mul_zero a).symm⟩

-- 1 divise tout nombre
theorem one_divides (a : Nat) : 1 | a :=
  ⟨a, (Nat.one_mul a).symm⟩

-- Reflexivite : a | a
theorem divides_refl (a : Nat) : a | a :=
  ⟨1, (Nat.mul_one a).symm⟩

### 7.2 Transitivite de la divisibilite

In [ ]:
-- Si a | b et b | c alors a | c
theorem divides_trans (a b c : Nat)
  (hab : a | b) (hbc : b | c) : a | c :=
  match hab, hbc with
  | ⟨k1, hk1⟩, ⟨k2, hk2⟩ =>
    ⟨k1 * k2, calc c = b * k2       := hk2
                   _ = (a * k1) * k2 := congrArg (· * k2) hk1
                   _ = a * (k1 * k2) := Nat.mul_assoc a k1 k2⟩

## 8. Exercices

### Exercice 1 : Prouver l'existence

In [ ]:
-- Prouver qu'il existe un nombre pair (divisible par 2)
theorem exists_even : \exists n : Nat, 2 | n := sorry

### Exercice 2 : Manipulation de forall

In [ ]:
-- Si P est vrai pour tout x et Q aussi, alors P /\ Q pour tout x
variable (A : Type) (P Q : A -> Prop)

theorem forall_and :
  (forall x : A, P x) -> (forall x : A, Q x) -> (forall x : A, P x /\ Q x) := sorry

### Exercice 3 : Interaction exists/forall

Cet exercice est plus avance et utilise plusieurs concepts :

1. **`[Inhabited A]`** : C'est une **typeclass** qui garantit que le type `A` possede au moins un element (appele `default`). Sans cette contrainte, un type pourrait etre vide, et l'implication `\neg(\forall x, P x) -> \exists x, \neg P x` ne serait pas prouvable (on ne pourrait pas exhiber de temoin).

2. **`byContradiction`** : Cette tactique (dans `Classical`) permet de prouver `P` en montrant que `\neg P` mene a une contradiction. C'est un principe de logique classique (pas disponible en logique constructive).

3. **Ordre des quantificateurs** : L'implication inverse `(\exists x, \neg P x) -> \neg(\forall x, P x)` est prouvable constructivement (sans `Classical`).

In [ ]:
-- Prouver que negation de forall implique exists negation (classique)
open Classical
variable (A : Type) [Inhabited A] (P : A -> Prop)

theorem not_forall_exists_not :
  (\neg forall x : A, P x) -> (\exists x : A, \neg P x) := sorry

## Solutions des exercices

In [ ]:
-- Solution exercice 1
theorem exists_even_sol : \exists n : Nat, 2 | n :=
  ⟨4, ⟨2, rfl⟩⟩   -- Temoin : 4, car 4 = 2 * 2

-- Solution exercice 2
variable (A : Type) (P Q : A -> Prop)

theorem forall_and_sol :
  (forall x : A, P x) -> (forall x : A, Q x) -> (forall x : A, P x /\ Q x) :=
  fun hp hq x => ⟨hp x, hq x⟩

-- Solution exercice 3
-- Note: Cette preuve utilise `byContradiction` qui est disponible via Classical.
-- `byContradiction` transforme un but `P` en `\neg P -> False`, permettant
-- une preuve par contradiction. C'est un principe de logique classique.
open Classical in
theorem not_forall_exists_not_sol
  (A : Type) [Inhabited A] (P : A -> Prop) :
  (\neg forall x : A, P x) -> (\exists x : A, \neg P x) :=
  fun h =>
    -- byContradiction : si supposer \neg(\exists x, \neg P x) mene a False,
    -- alors \exists x, \neg P x est vrai
    byContradiction fun hne =>
      -- Sous l'hypothese hne : \neg(\exists x, \neg P x)
      -- on peut montrer forall x, P x (par contraposition)
      have hall : forall x : A, P x := fun x =>
        -- Si P x etait faux, on aurait \exists x, \neg P x
        byContradiction fun hnp =>
          hne ⟨x, hnp⟩  -- Contradiction avec hne
      -- Mais h dit que \neg(forall x, P x)
      h hall  -- Contradiction finale

## Resume

| Concept | Syntaxe | Introduction | Elimination |
|---------|---------|--------------|-------------|
| **forall** | `\forall x : A, P x` | `fun x => ...` | Application `h a` |
| **exists** | `\exists x : A, P x` | `⟨temoin, preuve⟩` | `match h with \| ⟨x, hx⟩ => ...` |
| **have** | `have h : P := ...` | Lemme intermediaire | Reference par nom |
| **this** | Reference anonyme | Derniere hypothese | `this` |
| **‹P›** | Reference par type | Hypothese de type P | `‹P›` |

### Points cles

- `forall` est le Pi-type dans `Prop` - prouver = programmer une fonction
- `exists` necessite un temoin + une preuve
- L'ordre des quantificateurs est crucial
- Les lemmes de `Nat` sont utiles pour l'arithmetique

### Prochaine etape

Dans le notebook **Lean-5-Tactics**, nous decouvrirons le **mode tactique** qui permet de construire des preuves de maniere plus interactive, en decomposant les buts etape par etape.

---

*Notebook base sur "TP - Z3 - Tweety - Lean.pdf" Section VI.B.3 et adapte pour Lean 4*